### Realizar el flujo que se hizo en el primero archivo, pero ahora para el archivo language.csv

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [0]:
languagues_schemas = StructType([
    StructField("laguageId",IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])


In [0]:
languages_df = spark.read \
    .option("header", True) \
    .schema(languagues_schemas) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

In [0]:
languages_selection_df = languages_df.select("laguageId", "languageName")

In [0]:
languages_renamed_df = languages_selection_df.withColumnsRenamed({
    "laguageId": "language_id",
    "languageName": "language_name"
})

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_final_df = languages_renamed_df.withColumns({
    "ingestion_date": current_timestamp(),
    "enviroment": lit("production"),
    "file_date": lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.languages")

In [0]:
overwrite_partition(languages_final_df, "movie_silver", "languages", "file_date")

In [0]:
#languages_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/languages")
#languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages")

languages_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.languages")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/languages

In [0]:
%sql
select file_date, count(1)
from movie_silver.languages
group by file_date;

In [0]:
#display(spark.read.parquet("/mnt/datamoviehistory/silver/languages"))